# AWS Bedrock

## Text Summarization using Bedrock FM(s)

### Restart Kernel


In [ ]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

### Install Libraries

In [7]:
!pip install --upgrade pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 14.5 MB/s eta 0:00:00:00:01
  Attempting uninstall: pip
    Found existing installation: pip 23.3.1
    Uninstalling pip-23.3.1:
      Successfully uninstalled pip-23.3.1


In [2]:
%pip install --no-build-isolation --force-reinstall \
    "boto3>=1.28.57" \
    "awscli>=1.29.57" \
    "botocore>=1.31.57"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 548.2/548.2 kB 5.8 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.7/247.7 kB 2.9 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 1.8 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 32.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 48.8 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 705.5/705.5 kB 8.1 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 873.6 kB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 kB 1.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.9/84.9 kB 1.1 MB/s eta 0:00:00eta 0:00:01
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.1.0
    Uninstalling urllib3-2.1.0:
      Successfully uninstalled urllib3-2.1.0
  Attempting uninstall: six
    Found existing installation: six

In [ ]:
# %pip install  \
#     "langchain>=0.0.350" \
#     "transformers>=4.24,<5" \
#     sqlalchemy -U \
#     "faiss-cpu>=1.7,<2" \
#     "pypdf>=3.8,<4" \
#     pinecone-client==2.2.4 \
#     apache-beam==2.52. \
#     tiktoken==0.5.2 \
#     "ipywidgets>=7,<8" \
#     matplotlib==3.8.2


In [ ]:
!pip install PyPDF2


### Import Libraries

In [16]:
import json
import os
import sys
import pandas as pd

import boto3
import botocore
from IPython.display import display_markdown, Markdown, clear_output
from PyPDF2 import PdfReader


### Initialize boto session

In [12]:
# module_path = ".."
# sys.path.append(os.path.abspath(module_path))

boto_session = boto3.Session()
aws_region = boto_session.region_name
print(aws_region)
br_client = boto_session.client("bedrock", region_name=aws_region)
br_runtime = boto_session.client("bedrock-runtime", region_name=aws_region)


us-east-1


### Test Connection & List Foundation Models

In [18]:
fms = br_client.list_foundation_models()['modelSummaries']
dfFM = pd.DataFrame(fms)
print(dfFM.shape)
dfFM.head()

(45, 10)


,modelArn,modelId,modelName,providerName,inputModalities,outputModalities,responseStreamingSupported,customizationsSupported,inferenceTypesSupported,modelLifecycle
0,arn:aws:bedrock:us-east-1::foundation-model/am...,amazon.titan-tg1-large,Titan Text Large,Amazon,[TEXT],[TEXT],True,[],[ON_DEMAND],{'status': 'ACTIVE'}
1,arn:aws:bedrock:us-east-1::foundation-model/am...,amazon.titan-image-generator-v1:0,Titan Image Generator G1,Amazon,"[TEXT, IMAGE]",[IMAGE],NaN,[FINE_TUNING],"[ON_DEMAND, PROVISIONED]",{'status': 'ACTIVE'}
2,arn:aws:bedrock:us-east-1::foundation-model/am...,amazon.titan-image-generator-v1,Titan Image Generator G1,Amazon,"[TEXT, IMAGE]",[IMAGE],NaN,[],[ON_DEMAND],{'status': 'ACTIVE'}
3,arn:aws:bedrock:us-east-1::foundation-model/am...,amazon.titan-embed-g1-text-02,Titan Text Embeddings v2,Amazon,[TEXT],[EMBEDDING],NaN,[],[ON_DEMAND],{'status': 'ACTIVE'}
4,arn:aws:bedrock:us-east-1::foundation-model/am...,amazon.titan-text-lite-v1:0:4k,Titan Text G1 - Lite,Amazon,[TEXT],[TEXT],True,"[FINE_TUNING, CONTINUED_PRE_TRAINING]",[PROVISIONED],{'status': 'ACTIVE'}


In [20]:
dfFM.columns

Index(['modelArn', 'modelId', 'modelName', 'providerName', 'inputModalities',
       'outputModalities', 'responseStreamingSupported',
       'customizationsSupported', 'inferenceTypesSupported', 'modelLifecycle'],
      dtype='object')

In [19]:
dfFM.modelName.unique()

array(['Titan Text Large', 'Titan Image Generator G1',
       'Titan Text Embeddings v2', 'Titan Text G1 - Lite',
       'Titan Text G1 - Express', 'Titan Embeddings G1 - Text',
       'Titan Multimodal Embeddings G1', 'SDXL 0.8', 'SDXL 1.0',
       'J2 Grande Instruct', 'J2 Jumbo Instruct', 'Jurassic-2 Mid',
       'Jurassic-2 Ultra', 'Claude Instant', 'Claude', 'Command',
       'Command Light', 'Embed English', 'Embed Multilingual',
       'Llama 2 Chat 13B', 'Llama 2 Chat 70B', 'Llama 2 13B',
       'Llama 2 70B'], dtype=object)

## Text Summarization

### Download a public dataset

In [29]:
%%sh

wget -O fannie-mf-commentary-oct-2023.pdf https://www.fanniemae.com/media/49331/display

--2024-01-11 22:59:04--  https://www.fanniemae.com/media/49331/display
Resolving www.fanniemae.com (www.fanniemae.com)... 104.18.26.25, 104.18.27.25, 2606:4700::6812:1a19, ...
Connecting to www.fanniemae.com (www.fanniemae.com)|104.18.26.25|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 249442 (244K) [application/pdf]
Saving to: ‘fannie-mf-commentary-oct-2023.pdf’

     0K .......... .......... .......... .......... .......... 20% 13.6M 0s
    50K .......... .......... .......... .......... .......... 41% 13.4M 0s
   100K .......... .......... .......... .......... .......... 61% 39.4M 0s
   150K .......... .......... .......... .......... .......... 82%  214M 0s
   200K .......... .......... .......... .......... ...       100%  216M=0.009s

2024-01-11 22:59:04 (26.7 MB/s) - ‘fannie-mf-commentary-oct-2023.pdf’ saved [249442/249442]



### Read and Extract Text from PDF File

In [33]:
filename = 'fannie-mf-commentary-oct-2023.pdf'
reader = PdfReader(filename)
reader

In [39]:
pages = []

for idx, page in enumerate(reader.pages):
    print("Page ", idx + 1, "\n")
    text = page.extract_text(0) # 0 for orientation 90 degree upright 
    pages.append(text)
    print(text, "\n\n")

Page  1 

1Multifamily Economic and Market Commentary
OCTOBER 2023
Rising Number of Multifamily Properties Offering Concessions
Multifamily market fundamentals have softened in 2023 compared to the prior year, the result of mixed economic 
trends including slowing -but-still -positive job growth, elevated single -family housing prices keeping many renters in 
place, and continued favorable demographics. Rent growth was exceptional over the past two years, and, of course, 
unsustainable, thus 2023 has seen a substantial slowing of rent growth rates. There remains a robust pipeline of new 
apartment rental projects that are underway in the nation’s largest metros, and with recessionary concerns there has 
been a rise in the number of properties across the country offering concessions.
In the multifamily apartment rental market, concessions are incentives with an economic value for renters, such as 
periods of free rent, free utilities, or other amenities. And in a competitive market when

In [40]:
all_text = "\n".join(pages)
all_text

'1Multifamily Economic and Market Commentary\nOCTOBER 2023\nRising Number of Multifamily Properties Offering Concessions\nMultifamily market fundamentals have softened in 2023 compared to the prior year, the result of mixed economic \ntrends including slowing -but-still -positive job growth, elevated single -family housing prices keeping many renters in \nplace, and continued favorable demographics. Rent growth was exceptional over the past two years, and, of course, \nunsustainable, thus 2023 has seen a substantial slowing of rent growth rates. There remains a robust pipeline of new \napartment rental projects that are underway in the nation’s largest metros, and with recessionary concerns there has \nbeen a rise in the number of properties across the country offering concessions.\nIn the multifamily apartment rental market, concessions are incentives with an economic value for renters, such as \nperiods of free rent, free utilities, or other amenities. And in a competitive market whe

### Token Count

In [62]:
len(re.findall(r"[\w']+", all_text))

1674

### Prompt Engineering

In [68]:
# Do not add any information that is not mentioned in the text below.

prompt = f"""
Please provide a summary of the following text. 

<text>
{all_text}
</text>

"""

### LLM Parameters

In [69]:
body = json.dumps({"inputText": prompt, 
                   "textGenerationConfig":{
                       "maxTokenCount":256,
                       "stopSequences":[],
                       "temperature":0,
                       "topP":1
                   },
                  }) 

### Invoke Bedrock FM API

In [70]:
modelId = 'amazon.titan-tg1-large' # change this to use a different version from the model provider
accept = 'application/json'
contentType = 'application/json'

try:
    
    response = br_runtime.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
    response_body = json.loads(response.get('body').read())
    output_text = response_body.get('results')[0].get('outputText')
    print(len(re.findall(r"[\w']+", output_text)))
    print(output_text)

except botocore.exceptions.ClientError as error:    
    raise error

181
The number of multifamily properties offering concessions has increased in 2023 due to a mix of economic trends, including slowing but still positive job growth, elevated single-family housing prices, and favorable demographics. Rent growth has slowed significantly, and there is a robust pipeline of new apartment rental projects underway in the nation's largest metros. Concessions are incentives with an economic value for renters, such as periods of free rent, free utilities, or other amenities. The value of the concession being offered has slightly declined, from 7.7% of asking rent in August 2022 to 7.2% in August 2023. Supply of new units remains high, with nearly 730,000 units underway with completion dates expected in 2023, though only 164,000 were completed year to date through April.

The number of apartment units completed across the country may reach a record in 2023, with nearly 730,000 units underway with completion dates expected in 2023. New apartment supply has been c

In [47]:
# try:
#     response = br_runtime.invoke_model_with_response_stream(body=body, modelId=modelId, accept=accept, contentType=contentType)
#     stream = response.get('body')
#     output = list(stream)
#     output

# except botocore.exceptions.ClientError as error:
#     raise error

### Streaming LLM Output

In [49]:
try:
    response = br_runtime.invoke_model_with_response_stream(body=body, modelId=modelId, accept=accept, contentType=contentType)
    stream = response.get('body')
    output = []
    i = 1
    if stream:
        for event in stream:
            chunk = event.get('chunk')
            if chunk:
                chunk_obj = json.loads(chunk.get('bytes').decode())
                text = chunk_obj['outputText']
                clear_output(wait=True)
                output.append(text)
                display_markdown(Markdown(''.join(output)))
                i+=1

    clear_output(wait=True)
    print(''.join(output))

except botocore.exceptions.ClientError as error:
    raise error

Here is a summary of the article:

The multifamily market has softened in 2023 due to mixed economic trends, including slowing but still positive job growth, elevated single-family housing prices, and continued favorable demographics. Rent growth has slowed significantly, and there is a robust pipeline of new apartment rental projects underway in the nation's largest metros. Concessions are incentives with an economic value for renters, such as periods of free rent, free utilities, or other amenities. The number of units offering concessions has increased from 5.2% of all units in August 2022 to 9.0% in August 2023, but the value of the concession being offered has slightly declined, from 7.7% of asking rent in August 2022 to 7.2% in August 2023. The supply of new apartment units completed across the country may reach a record in 2023, with nearly 730,000 units underway with completion dates expected in 2023.

New apartment supply has been concentrated in the nation's largest and rapid

In [ ]:
# page total 
# Accelerator title
# Summary 
# Signature
# 


# 
# LangChain
